# Identifying critical points of a callback function
This is my current main function:

```py
    def main(self) -> None:
        def callback(indata: npt.NDArray[DTYPE], outdata: npt.NDArray[DTYPE],
                     frames: int, time: Any, status: sd.CallbackFlags) -> None:
            if status:
                print(status)

            if not self._tracks_lock.locked():
                tracks = self._tracks
            else:
                tracks = self._tracks_copy

            if self._recording:
                self._recorded_track = np.concatenate(
                    [self._recorded_track, indata])

            sliced_data = [indata]
            # slice
            for track in tracks:
                start = self._current_frame % len(track)
                end = (self._current_frame+frames) % len(track)
                if end < start:
                    track_slice = np.concatenate(
                        (track[start:], track[:end]))
                else:
                    track_slice = track[start:end]
                sliced_data.append(track_slice)

            mixed_data = np.mean(a=sliced_data, axis=0)

            outdata[:] = mixed_data

            self._current_frame += frames

        with sd.Stream(samplerate=SAMPLERATE, blocksize=BLOCKSIZE, dtype=STR_DTYPE,
                       channels=CHANNELS, callback=callback) as stream:
            while self._stream_active:
                sleep(1)
```
Lets time it snippet by snippet.

## If status
I will skip this one for now, as it is gonna be replaced by error handling soon.